In [ ]:
# Análisis de PP - Plume Network
#1. Obtener datos de wallets con XP > 0 
#2. Clasificarlas en 5 rangos de XP (tiers)
#3. Calcular:
#   - XP total/promedio/mín/máx
#   - Valor estimado de 1 XP en terminos de PLUME

   
# Instalar dependencias (si es necesario)
# !pip install requests --quiet

import requests
import pandas as pd

# Configuración API
BASE_URL = "https://portal-api.plume.org/api/v1/stats/leaderboard"
COUNT_PER_PAGE = 2_000  # Máximo de wallets por consulta
TIMEOUT_SECONDS = 30
PLUME_SUPPLY_S2 = 150_000_000  # Total de tokens para el airdrop S2

# Definición de Tiers (rangos de XP)
tiers = [
    (0, 999, "0 – 0.99k"),
    (1_000, 19_999, "1k – 19.99k"),
    (20_000, 29_999, "20k – 29.99k"),
    (30_000, 49_999, "30k – 49.99k"),
    (50_000, float("inf"), "50k+"),
]

def asignar_tier(xp: int) -> str:
    """Clasifica una wallet en su tier correspondiente según su XP."""
    for min_xp, max_xp, etiqueta in tiers:
        if min_xp <= xp <= max_xp:
            return etiqueta
    return "—"

# Recolección de datos
offset = 0
wallets_unicas = set()
xp_total = 0
xp_min = float("inf")
xp_max = 0
conteo_tiers = {etiqueta: 0 for _, _, etiqueta in tiers}

while True:
    params = {
        "offset": offset,
        "count": COUNT_PER_PAGE,
        "walletAddress": "undefined",
        "overrideDay1Override": "false",
        "preview": "false",
    }

    respuesta = requests.get(BASE_URL, params=params, timeout=TIMEOUT_SECONDS)
    respuesta.raise_for_status()
    datos_pagina = respuesta.json().get("data", {}).get("leaderboard", [])

    if not datos_pagina:  # Fin cuando no hay más datos
        break

    for entrada in datos_pagina:
        xp = entrada.get("totalXp", 0)
        wallet = entrada.get("walletAddress")

        if xp == 0:  # Fin al encontrar primer XP cero
            datos_pagina = []
            break

        if wallet not in wallets_unicas:
            wallets_unicas.add(wallet)
            xp_total += xp
            xp_min = min(xp_min, xp)
            xp_max = max(xp_max, xp)
            conteo_tiers[asignar_tier(xp)] += 1

    offset += COUNT_PER_PAGE
    if not datos_pagina:
        break

# Cálculos finales
n_wallets = len(wallets_unicas)
xp_promedio = xp_total / n_wallets
valor_punto = PLUME_SUPPLY_S2 / xp_total  # Tokens PLUME por 1 XP

# Resultados
print(f"🔹 Wallets únicas: {n_wallets:,}")
print(f"🔹 XP total acumulado: {xp_total:,}")
print(f"🔹 XP promedio por wallet: {xp_promedio:,.2f}")
print(f"🔹 XP mínimo detectado: {xp_min:,}")
print(f"🔹 XP máximo detectado: {xp_max:,}")
print(f"🔹 Valor estimado por 1 XP: {valor_punto:.8f} PLUME")

# Tabla de distribución por tiers
df_tiers = pd.DataFrame([
    {"Tier": tier, "Wallets": count, "% del total": (count/n_wallets)*100}
    for tier, count in conteo_tiers.items()
]).sort_values("Tier")

display(df_tiers)

🔹 Wallets únicas: 202,942
🔹 XP total acumulado: 662,344,005
🔹 XP promedio por wallet: 3,263.71
🔹 XP mínimo detectado: 50
🔹 XP máximo detectado: 139,269
🔹 Valor estimado por 1 XP: 0.22646842 PLUME


,Tier,Wallets,% del total
0,0 – 0.99k,92601,45.629293
1,1k – 19.99k,107559,52.999872
2,20k – 29.99k,2222,1.094894
3,30k – 49.99k,514,0.253274
4,50k+,46,0.022667


In [1]:
# Top 3 actividades que mas puntos dieron (de ayer a hoy)
# Toma las wallets del laderboard
# Las pasa a la api pp totals
# Hace el conteo de cuantas veces aparece x accion en el podio de cada wallet

# # Instalar dependencias (si es necesario)
#!pip install aiohttp nest_asyncio pandas tqdm --quiet

# Configuración
LEADERBOARD_URL = "https://portal-api.plume.org/api/v1/stats/leaderboard"
PP_TOTALS_URL   = "https://portal-api.plume.org/api/v1/stats/pp-totals"
LB_BATCH_SIZE   = 10_000 
CONCURRENCY     = 30         
TIMEOUT_SECS    = 30
HEADERS         = {"User-Agent": "plume-fast-scan/1.0"}

import math, asyncio, aiohttp, nest_asyncio, collections, pandas as pd
from tqdm.auto import tqdm
nest_asyncio.apply()

# Obtener las wallets con XP>0
async def fetch_leaderboard_wallets():
    wallets = []
    offset  = 0
    async with aiohttp.ClientSession(headers=HEADERS) as session:
        while True:
            params = {
                "offset": offset,
                "count":  LB_BATCH_SIZE,
                "walletAddress": "undefined",
                "overrideDay1Override": "false",
                "preview": "false",
            }
            async with session.get(LEADERBOARD_URL, params=params, timeout=TIMEOUT_SECS) as r:
                page = (await r.json())["data"]["leaderboard"]
            if not page:
                break
            for row in page:
                if row["totalXp"] == 0:               
                    return wallets
                wallets.append(row["walletAddress"].lower())
            offset += LB_BATCH_SIZE
    return wallets

wallets = await fetch_leaderboard_wallets()
print(f"Total wallets XP>0 encontradas: {len(wallets):,}")

# Consultar pp-totals de todas las wallets
sem      = asyncio.Semaphore(CONCURRENCY)
counter  = collections.Counter()

async def fetch_pp_totals(session, wallet):
    url = f"{PP_TOTALS_URL}?walletAddress={wallet}"
    async with sem, session.get(url, timeout=TIMEOUT_SECS) as resp:
        js = await resp.json()
    top3 = js.get("data", {}).get("ppScores", {}).get("top3PointsDeltasStrings", [])
    return top3

async def gather_all(wallet_list):
    async with aiohttp.ClientSession(headers=HEADERS) as session:
        tasks = [fetch_pp_totals(session, w) for w in wallet_list]
        for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="pp‑totals"):
            top3 = await fut
            counter.update(top3)

await gather_all(wallets)

# Resultados
df = (
    pd.DataFrame(counter.items(), columns=["Acción", "Veces_en_top3"])
      .sort_values("Veces_en_top3", ascending=False)
      .reset_index(drop=True)
)
display(df)


/home/ismael/Desktop/wallet_autopsy/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total wallets XP>0 encontradas: 202,942


pp‑totals: 100%|██████████| 202942/202942 [12:06<00:00, 279.16it/s]


,Acción,Veces_en_top3
0,protocol-interactions,37967
1,staking,18546
2,referrals,4340
3,trades,1980
4,quests,1797
5,tvl,1568
6,bridging,1100
7,sky-society,173


In [3]:
# PLUME posibles sybils
# 1) Descargar todo el leaderboard (hasta XP == 0).
# 2) Filtra wallets cuyo totalXp esté en {50, 100, 150, …, 900}.
# 3) Cuenta cuántas hay de cada valor.

# Instalar dependencias (si es necesario)
#!pip install aiohttp nest_asyncio pandas tqdm --quiet

#Configuración
LEADERBOARD_URL = "https://portal-api.plume.org/api/v1/stats/leaderboard"
BATCH_SIZE      = 10_000      # filas por llamada
TIMEOUT_SECS    = 30
HEADERS         = {"User-Agent": "plume-xp-snapshot/1.0"}

# XP que queremos rastrear
XP_TARGETS = {xp for xp in range(50, 1000, 50)}  

import asyncio, aiohttp, nest_asyncio, pandas as pd
from collections import defaultdict
from tqdm.auto import tqdm
nest_asyncio.apply()

#Descargar leaderboard completo
async def fetch_all_wallets():
    wallets_by_xp = defaultdict(list)
    offset = 0

    async with aiohttp.ClientSession(headers=HEADERS) as session:
        while True:
            params = {
                "offset": offset,
                "count":  BATCH_SIZE,
                "walletAddress": "undefined",
                "overrideDay1Override": "false",
                "preview": "false",
            }
            async with session.get(LEADERBOARD_URL, params=params,
                                   timeout=TIMEOUT_SECS) as r:
                page = (await r.json())["data"]["leaderboard"]

            if not page:
                break

            for row in page:
                xp = row["totalXp"]
                if xp == 0:
                    return wallets_by_xp
                if xp in XP_TARGETS:
                    wallets_by_xp[xp].append(row["walletAddress"].lower())

            offset += BATCH_SIZE

    return wallets_by_xp

wallets_by_xp = await fetch_all_wallets()

# Resumen de conteos
print("=== Conteo de wallets por PP objetivo ===")
for xp in sorted(XP_TARGETS):
    cnt = len(wallets_by_xp.get(xp, []))
    print(f"PP {xp:>3}: {cnt:,} wallets")

=== Conteo de wallets por PP objetivo ===
PP  50: 1,043 wallets
PP 100: 4,355 wallets
PP 150: 134 wallets
PP 200: 39,045 wallets
PP 250: 446 wallets
PP 300: 9,289 wallets
PP 350: 546 wallets
PP 400: 18 wallets
PP 450: 0 wallets
PP 500: 10,594 wallets
PP 550: 324 wallets
PP 600: 17 wallets
PP 650: 0 wallets
PP 700: 25,648 wallets
PP 750: 628 wallets
PP 800: 19 wallets
PP 850: 3 wallets
PP 900: 96 wallets
PP 950: 294 wallets


In [4]:
# Análisis de wallets selectas para estrategia - Plume Network
#1. Obtener datos del leaderboard
#2. Analizar 4 segmentos clave:
#   - Top 1-100
#   - Top 500-1000  
#   - Top 1000-2000
#   - Top 2000-3000
#3. Para cada segmento:
#   - Extraer top 20 wallets
#   - Mostrar datos (rank, XP)
#4. Generar lista consolidada de wallets

# Instalar dependencias (si es necesario)
# !pip install requests pandas tqdm --quiet
import requests
import pandas as pd
from tqdm import tqdm

LB_URL = "https://portal-api.plume.org/api/v1/stats/leaderboard"
COUNT_PER_PAGE = 20_000
TIMEOUT_SECONDS = 30
HEADERS = {'User-Agent': 'Plume-Wallet-Analyzer/1.0'}

# Obtener Datos
def get_leaderboard_data(offset):
    params = {
        "offset": offset,
        "count": COUNT_PER_PAGE,
        "walletAddress": "undefined",
        "overrideDay1Override": "false",
        "preview": "false"
    }
    try:
        response = requests.get(LB_URL, params=params, 
                              headers=HEADERS, timeout=TIMEOUT_SECONDS)
        response.raise_for_status()
        return response.json().get("data", {}).get("leaderboard", [])
    except Exception as e:
        print(f"Error al obtener datos: {e}")
        return None

# Procesamiento Principal
def analyze_wallets():
    all_wallets = []
    offset = 0
    
    print("🔍 Recopilando datos del leaderboard...")
    
    with tqdm(desc="Progreso", unit=" páginas") as progress_bar:
        while True:
            data = get_leaderboard_data(offset)
            if not data:
                break
                
            for wallet in data:
                if wallet.get("totalXp", 0) == 0:
                    break
                
                if wallet.get("referralBonusXp", 0) == 0:
                    all_wallets.append({
                        'Rank': wallet["xpRank"],
                        'Wallet': wallet["walletAddress"],
                        'Self XP': wallet.get("userSelfXp", 0),
                        'Total XP': wallet.get("totalXp", 0)
                    })
            
            if data[-1].get("totalXp", 0) == 0:
                break
                
            offset += COUNT_PER_PAGE
            progress_bar.update(1)
    
    if not all_wallets:
        print("❌ No se encontraron wallets válidas")
        return
    
    df = pd.DataFrame(all_wallets) 
    segments = {
        'Top 1-100': (1, 100),
        'Top 500-1000': (500, 1000),
        'Top 1000-2000': (1000, 2000),
        'Top 2000-3000': (2000, 3000)
    }
    
    results = {}
    for name, (start, end) in segments.items():
        segment = df[(df['Rank'] >= start) & (df['Rank'] <= end)]
        results[name] = segment.nlargest(20, 'Self XP') if name == 'Top 1-100' else segment.head(20)
    all_selected_wallets = []
    
    for segment_name, segment_data in results.items():
        print(f"\n {segment_name} - Top 20 Wallets")
        display(segment_data)
        all_selected_wallets.extend(segment_data['Wallet'].tolist())
    print("\n Lista completa de wallets seleccionadas:")
    print(",\n".join(all_selected_wallets))

if __name__ == "__main__":
    analyze_wallets()

🔍 Recopilando datos del leaderboard...


Progreso: 10 páginas [00:40,  4.09s/ páginas]



 Top 1-100 - Top 20 Wallets


,Rank,Wallet,Self XP,Total XP
0,1,0xff0c6444cb0fa6121a85e838219354bfe2e1556b,139269,139269
1,2,0x4ceaed0b2aea8bcce21070cdc1bf8d4057c0e5b4,121621,121621
2,3,0x861d7d7c8127794236f62bdb98846413c99d29b8,87826,87826
3,5,0x1851a4dd5c5cc7242423538ef288db6e00782e2a,84331,84331
4,9,0x7cd4f4b2b811cf35559caa1d85f2097f38f882bb,75904,75904
5,10,0x07ff87d3a7a1ef3e096a05ac498ef45ea6991b78,75317,75317
6,12,0x70ddd8ffc952e4c46bc8aa5b3e0b02be93b89eca,70393,70393
7,13,0xfacc8546eda192b7461cbdde122ebc1ff998311d,69875,69875
8,15,0x8869f48507ce2ca834b7c75893f75d0af8811d0d,65500,65500
9,19,0xfb61f6df17f0796f8313c20600e82eebf8e2d030,62192,62192



 Top 500-1000 - Top 20 Wallets


...


 Top 1000-2000 - Top 20 Wallets


,Rank,Wallet,Self XP,Total XP
369,1001,0xdf12817239526e111f6cc3a8ac0c3650305027a8,26119,26119
370,1003,0x5be5ee7293145d1c833e2fce00efeda88b69738b,26110,26110
371,1004,0x0fea1758b2e66ff01dd34dfd244b15a87b6c97ce,26083,26083
372,1007,0xca484677c636a990ddbf28e968f46c010029b699,26047,26047
373,1011,0x8efec084cbcead96bb719142589a29bbb5d781dc,26034,26034
374,1012,0xe11fc7ccbdd92ec286a439b8aaffe74140ac7f9a,26034,26034
375,1015,0x17a329c250221586fec7626fcb12eb30b52eff9d,26007,26007
376,1017,0x947db9d96aea1650d4879eba95e0d6aa0dd93e90,26001,26001
377,1020,0xddc1c7957733e07560965543702814138fa34d6e,25987,25987
378,1021,0x6304cf7c429cdfea96dcd218fb91a7ea38dd05f6,25972,25972



 Top 2000-3000 - Top 20 Wallets


,Rank,Wallet,Self XP,Total XP
898,2000,0x75e55f3a72aa8381d1e01cd06db53df15e06a6ae,21748,21748
899,2001,0xc467972a2fa614c5f6990968d9d47b70db169f5e,21745,21745
900,2003,0x8364ab71eaabe46fd130a9b995082014138a3a36,21744,21744
901,2005,0xb74cce4b9dfd34f6d2217f403ec3c19026414b64,21742,21742
902,2006,0xfba7cf91cef3fb21455f6c1ea1c8d16e804a1faf,21739,21739
903,2008,0x6f8ceb284daf956d03dfc8696ceb41cdd234f175,21735,21735
904,2010,0x628ea720fe8abe7f8c150936fd87dbc20ad887d6,21728,21728
905,2011,0xb2dd9d30e60aa1e48875c74ba450ca938e3af408,21727,21727
906,2012,0x2c613881b4a64cfd419c2b19ac3307e7cf05cd0d,21726,21726
907,2013,0x7e0371ec1177d3cf177a1a9a87dbc1e1864cd828,21725,21725



 Lista completa de wallets seleccionadas:
0xff0c6444cb0fa6121a85e838219354bfe2e1556b,
0x4ceaed0b2aea8bcce21070cdc1bf8d4057c0e5b4,
0x861d7d7c8127794236f62bdb98846413c99d29b8,
0x1851a4dd5c5cc7242423538ef288db6e00782e2a,
0x7cd4f4b2b811cf35559caa1d85f2097f38f882bb,
0x07ff87d3a7a1ef3e096a05ac498ef45ea6991b78,
0x70ddd8ffc952e4c46bc8aa5b3e0b02be93b89eca,
0xfacc8546eda192b7461cbdde122ebc1ff998311d,
0x8869f48507ce2ca834b7c75893f75d0af8811d0d,
0xfb61f6df17f0796f8313c20600e82eebf8e2d030,
0x3eea372582e72f1d189ce02e7a65968015622a01,
0x8ccf6fa07a116ef81b5b742416190b33aee5bfe2,
0xd7583e3cf08bbcab66f1242195227bbf9f865fda,
0x20089381f65568cbaace03f5e5cd920b3ee7b6f5,
0xdb371cf6f519375e51995dbe0f537e291b099c9b,
0x01cb0c5e571e1e3fda1ff7c1118f37351616b9fe,
0xbd73cf5baf12f120ee3f6c4ad82df9a12649e578,
0x7eafcc697e7bc14313c22325ecc17b21d5232bf9,
0x0a0970382ce3d4ddabbcd5e4ccb4a4c8dbe50641,
0xcfe077e6f7554b1724546e02624a0832d1f4557a,
0x0c1856cb7444cc8596c706185b54535f45c2623b,
0x9342174252e8ae1de46abefc1b930e2

In [5]:
# Análisis de Interacciones con Contratos - Plume Network

#1. Analizar 100+ wallets predefinidas (en el codigo anterior)
#2. Identificar:
#   - Contratos más interactuados
#   - Contratos compartidos entre wallets
#3. Calcular métricas de uso
#  - Frecuencia por contrato
#  - Wallets por contrato
#  - Estadísticas globales
#En este codigo hacemos uso de las apis del explorer

# Instalar dependencias (si es necesario)
# !pip install requests collections tqdm 
import requests
from collections import Counter
from tqdm import tqdm

class PlumeGlobalContractAnalyzer:
    def __init__(self):
        self.base_url = "https://explorer.plume.org/api"
        self.wallets = [
            '0xff0c6444cb0fa6121a85e838219354bfe2e1556b','0x4ceaed0b2aea8bcce21070cdc1bf8d4057c0e5b4',
            '0x861d7d7c8127794236f62bdb98846413c99d29b8','0x1851a4dd5c5cc7242423538ef288db6e00782e2a',
            '0x7cd4f4b2b811cf35559caa1d85f2097f38f882bb','0x07ff87d3a7a1ef3e096a05ac498ef45ea6991b78',
            '0xfacc8546eda192b7461cbdde122ebc1ff998311d','0x70ddd8ffc952e4c46bc8aa5b3e0b02be93b89eca',
            '0x8869f48507ce2ca834b7c75893f75d0af8811d0d','0xfb61f6df17f0796f8313c20600e82eebf8e2d030',
            '0x8ccf6fa07a116ef81b5b742416190b33aee5bfe2','0x3eea372582e72f1d189ce02e7a65968015622a01',
            '0xd7583e3cf08bbcab66f1242195227bbf9f865fda','0x20089381f65568cbaace03f5e5cd920b3ee7b6f5',
            '0xdb371cf6f519375e51995dbe0f537e291b099c9b','0x01cb0c5e571e1e3fda1ff7c1118f37351616b9fe',
            '0x7eafcc697e7bc14313c22325ecc17b21d5232bf9','0xbd73cf5baf12f120ee3f6c4ad82df9a12649e578',
            '0x0a0970382ce3d4ddabbcd5e4ccb4a4c8dbe50641','0xcfe077e6f7554b1724546e02624a0832d1f4557a',
            '0xa41c88c77c339e46416609da258b345c999eeaf2','0x26b9c4a911b68d33414b459bc4d51a988615587a',
            '0x748225805ddf43f73d8ef6815945f577f3139bd9','0x32d81b31f8674df5f72f99802346da263c3ffe29',
            '0x5dbc78b8f42e62bc61b4976d6c77a16cb96ab074','0xd3b34dd52ca4c7c33d278208f8391c4fba887e00',
            '0x25d12fe10793d46ab4dfe8e62cf31bc86783596c','0x3d0bc8eca7bf9e01b00b4270cdb34505808b8a19',
            '0x7d6d466d0beaf3ffb11a9e9253672a8d41a1e00a','0xf75729dca43e96dc38f519d1dbefb950ae95a507',
            '0x9e125909dcada892fb48e82070b8d44d9cd05b095','0xb922940e9d5e5fc6a3c63527c9b02c7070ba6b29',
            '0x2f6a4152abffae4434e6893777ee23f44291a10e','0x9f7605a1bab5eb1a19700657637701c38437e300',
            '0x3aa50e86b3ac589bf3a9b9d3f90bb6801611e8ed','0xe9397eac529721910a2871230203c0b9270bdb9b',
            '0xa371f994981b5ab257880396296b5bdbfb2eebe3','0x8ae4c29c97a9dfba7a4440c6336a7d37eba0e12c',
            '0x71a19a2cb659fbd0e52b6bc1d9a72924d0a0d713','0x24c85830af787a6d243fb45c1ec5a57612bc6623',
            '0xa97442113e0190b9a2cd839ddd90a671f734a911','0x2d2e13eeda4cc6263eab5af2fbebb6e9d341dfd0',
            '0xddc1c7957733e07560965543702814138fa34d6e','0x8efec084cbcead96bb719142589a29bbb5d781dc',
            '0x0fea1758b2e66ff01dd34dfd244b15a87b6c97ce','0x5be5ee7293145d1c833e2fce00efeda88b69738ba',
            '0xc987b515dea2a38509f7344cabb4dddfe0d29958','0x4819c90edf7f46bb7ac74b1b6ccff010040ca92c',
            '0xe11fc7ccbdd92ec286a439b8aaffe74140ac7f9a','0x6304cf7c429cdfea96dcd218fb91a7ea38dd05f6',
            '0x02ecebe7f99537a05a1295e7a4593d496d327a65','0xdf12817239526e111f6cc3a8ac0c3650305027a8',
            '0xd771e188598fe2a7ee88e5676da643a9c0ef279c','0xf254d919f728ceb4c29f03b3f5e665baf35bc8e4',
            '0x105347f720fd0aefb4470e4bf352217301313564','0xea5038de39d2ae7d20c0c675212f41c6958ab6f3',
            '0xca484677c636a990ddbf28e968f46c010029b699','0xbe837ff72f22ee92931b87db449dca9ca7432906',
            '0xa63fda5bea073075f204e42bedb78ed882fae44c','0x5502e6aebbc9d99abdba90348344fcb01539e567',
            '0x77241bd1bc57e4116f49a742c53b74695fd58277','0xeea8f6dd11c44115dec8d5640eab3771980a2086',
            '0x7c70686088221ee68d2d66bf6667105fbb01908c','0xe35836f7cf3a764ce3f9e7cf1f822d09d07f1f90',
            '0xb1606c9c4906dc522b8d445ca866b353fb70ca89','0x580ab45478cac2d56d58b02f1a87aa73b80001ce',
            '0x2e7c90cc68b3d4c84f1d5305f06920c38cbdeff1','0x17f4fb9217eac09d7da8a8b7e638f9658cc4fc8f',
            '0x5379414745f63da71c0c3ae79441a27c1759f465','0xefe6c7271627eac3b47008112a38695b4e76984c',
            '0xb74cce4b9dfd34f6d2217f403ec3c19026414b64','0xdbe452f1a0d0190e161b5fae8ee3a530f099d9a3',
            '0x03a1697ebc95fadb5616d763bb4b475a9bb38151','0xb0c9aeb8ec42ec9beb1fbaedb76aaca93208b5f9',
            '0x1871011aa4b00e8e0f10b21dbf4ef933f9badfac','0x09bcab589d27109e95d7b1ca56861b3a91f7246f',
            '0xc467972a2fa614c5f6990968d9d47b70db169f5e','0x8364ab71eaabe46fd130a9b995082014138a3a36',
            '0xf0c3e412f497ed311cea84d73127697ad761a2aa','0xf40d9019940de42f44fc67fbd6f7e7637401595f'
        ]
        self.all_contracts = Counter()
        self.contract_to_wallets = {}

    def fetch_transactions(self, address, max_retries=3):
        params = {'module': 'account', 'action': 'txlist', 'address': address}
        
        for attempt in range(max_retries):
            try:
                response = requests.get(self.base_url, params=params, timeout=15)
                response.raise_for_status()
                data = response.json()
                return data.get('result', []) if data.get('message') == 'OK' else []
            except Exception as e:
                print(f"Intento {attempt+1} fallido para {address[:8]}...: {str(e)[:50]}...")
                if attempt == max_retries - 1:
                    return []
        return []

    def process_wallet(self, wallet_address):
        transactions = self.fetch_transactions(wallet_address)
        
        for tx in transactions:
            if len(tx.get('input', '')) > 2: 
                contract = tx.get('to', '').lower()
                if contract:
                    self.all_contracts.update([contract])
                    if contract not in self.contract_to_wallets:
                        self.contract_to_wallets[contract] = set()
                    self.contract_to_wallets[contract].add(wallet_address)

    def run_analysis(self):
        """Ejecuta el análisis completo y muestra resultados globales"""
        print("Analizando interacciones con contratos...")
        for wallet in tqdm(self.wallets, desc="Procesando wallets"):
            self.process_wallet(wallet)
        self.show_global_results()

    def show_global_results(self):
        """Muestra el análisis consolidado de todos los contratos"""
        total_interactions = sum(self.all_contracts.values())
        unique_contracts = len(self.all_contracts)
        
        print(f"\nRESULTADOS GLOBALES ({len(self.wallets)} wallets analizadas)")
        print("="*60)
        print("\nTOP 20 CONTRATOS MÁS UTILIZADOS:")
        for contract, count in self.all_contracts.most_common(20):
            print(f"  • {contract}: {count} interacciones")
        shared_contracts = {k: len(v) for k, v in self.contract_to_wallets.items() if len(v) > 1}
        print("\nTOP 10 CONTRATOS COMPARTIDOS (usados por múltiples wallets):")
        for contract, wallet_count in sorted(shared_contracts.items(), 
                                          key=lambda x: x[1], reverse=True)[:10]:
            print(f"  • {contract}: usado por {wallet_count} wallets")
        print("\n ESTADÍSTICAS:")
        print(f"  • Total interacciones con contratos: {total_interactions}")
        print(f"  • Contratos únicos interactuados: {unique_contracts}")
        print(f"  • Promedio interacciones por contrato: {total_interactions/unique_contracts:.1f}")
        usage_distribution = Counter()
        for wallets in self.contract_to_wallets.values():
            usage_distribution[len(wallets)] += 1
if __name__ == "__main__":
    analyzer = PlumeGlobalContractAnalyzer()
    analyzer.run_analysis()

Analizando interacciones con contratos...


Procesando wallets: 100%|██████████| 80/80 [00:41<00:00,  1.95it/s]


RESULTADOS GLOBALES (80 wallets analizadas)

TOP 20 CONTRATOS MÁS UTILIZADOS:
  • 0xdddd73f5df1f0dc31373357beac77545dc5a6f3f: 4368 interacciones
  • 0xea237441c92cae6fc17caaf9a7acb3f953be4bd1: 3009 interacciones
  • 0x35e44dc4702fd51744001e248b49cbf9fcc51f0c: 2806 interacciones
  • 0xaaaaaaaa81a99d2a05ee428ec7a1d8a3c2237d85: 2707 interacciones
  • 0xb8e9f72bd52575b61705e4376970e14c9f573d71: 2165 interacciones
  • 0x6104fe10ca937a086ba7adbd0910a4733d380cb6: 1546 interacciones
  • 0x30c791e4654edac575fa1700ed8633cb2fede871: 1215 interacciones
  • 0x36ec4d4ecfa1e40c055ad4c7cf9085057971ae68: 1105 interacciones
  • 0x42b18785ce0aed7bf7ca43a39471ed4c0a3e0bb5: 992 interacciones
  • 0xc0df5784f28046d11813356919b869dda5815b16: 907 interacciones
  • 0x78add880a697070c1e765ac44d65323a0dcce913: 859 interacciones
  • 0xd8f185769b6e2918b759e83f7ec268c882800ec7: 751 interacciones
  • 0x593ccca4c4bf58b7526a4c164ceef4003c6388db: 655 interacciones
  • 0x77ab297da4f3667059ef0c32f5bc657f1006cbb0: 580 int